In [1]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tqdm
import shutil
import warnings
warnings.filterwarnings('ignore')

def show_subplot_img(img_list, size = 10):
    fig = plt.figure(figsize=(size,size))

    for n in range(len(img_list)):
        ax = fig.add_subplot(1,len(img_list),n+1)
        ax.imshow(cv2.cvtColor(img_list[n], cv2.COLOR_BGR2RGB),cmap="gray")
        ax.axis('off')
    plt.show()

def validate_dataset(df, img_dir):
    count = 0
    for rows in df.itertuples():
        if os.path.isfile(img_dir+'/'+ rows.id):
            continue
        else:
            count += 1
            df.drop(df[df['id'] == rows.id].index, inplace=True)
    print("Not founded images (Num) : ",count)
    return df

In [2]:
base_path = './data/images/'
target_path = './data/images2/'
os.makedirs(target_path, exist_ok=True)

df = pd.read_csv('./data/' + "result.csv")
df = validate_dataset(df=df, img_dir=base_path)
col_list = df.columns
col_list

Not founded images (Num) :  24


Index(['id', 'good', 'b_edge', 'burr', 'borken', 'b_bubble', 'etc', 'no_lens'], dtype='object')

In [3]:
print(len(df))
df['sum'] = df[['good', 'b_edge', 'burr', 'borken', 'b_bubble', 'etc', 'no_lens']].sum(axis=1)
df.drop(df[df['sum'] == 0].index, inplace=True)
print(len(df))

129851
108823


In [11]:
df.sum()

id          01-2.jpg02-2.jpg03-2.jpg04-2.jpg05-2.jpg202210...
good                                                   103910
b_edge                                                    786
burr                                                      249
borken                                                    329
b_bubble                                                 1289
etc                                                      2381
no_lens                                                     0
sum                                                    108944
dtype: object

In [5]:
wrong_df = df.loc[(df['good'] == 1) & (df['sum']>1)]
wrong_df


,id,good,b_edge,burr,borken,b_bubble,etc,no_lens,sum


In [6]:
b_edge_df = df[df['b_edge']==1]
burr_df = df[df['burr']==1]
borken_df = df[df['borken']==1]
etc_df = df[df['etc']==1]

print(len(b_edge_df))
print(len(burr_df))
print(len(borken_df))
print(len(etc_df))


786
249
329
2381


In [7]:
new_df = pd.DataFrame(columns=col_list)
save_img = True
be_bar = tqdm.tqdm(df.itertuples(), total=len(df))
for rows in be_bar:
    if rows.b_edge > 0 or rows.burr > 0 or rows.borken > 0 or rows.b_bubble >0:
        new_row = pd.Series([rows.id, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)
        img = cv2.imread(base_path + rows.id)
        if os.path.isfile(target_path + rows.id) is False and save_img:
            shutil.copy(base_path + rows.id, target_path + rows.id)
        
        img_flip_lr = cv2.flip(img, 1)
        lr_filename = 'lr'+rows.id
        if os.path.isfile(target_path + lr_filename) is False and save_img:
            cv2.imwrite(target_path + lr_filename ,img_flip_lr)
        new_row = pd.Series([lr_filename, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)

        img_flip_ud = cv2.flip(img, 0)
        ud_filename = "ud" + rows.id
        if os.path.isfile(target_path + ud_filename) is False and save_img:
            cv2.imwrite(target_path + ud_filename ,img_flip_ud)
        new_row = pd.Series([ud_filename, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)

        img_flip_lrud = cv2.flip(img_flip_lr, 0)
        lrud_filename = 'lrud' + rows.id
        if os.path.isfile(target_path + lrud_filename) is False and save_img:
            cv2.imwrite(target_path + lrud_filename ,img_flip_lrud)
        new_row = pd.Series([lrud_filename, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)
    elif rows.etc > 0:
        new_row = pd.Series([rows.id, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
        new_df = new_df.append(new_row, ignore_index=True)
        img = cv2.imread(base_path + rows.id)
        if os.path.isfile(target_path + rows.id) is False and save_img:
            shutil.copy(base_path + rows.id, target_path + rows.id)


new_df

100%|██████████| 108823/108823 [06:48<00:00, 266.64it/s] 


,id,good,b_edge,burr,borken,b_bubble,etc,no_lens
0,01-2.jpg,0,0,0,0,1,0,0
1,lr01-2.jpg,0,0,0,0,1,0,0
2,ud01-2.jpg,0,0,0,0,1,0,0
3,lrud01-2.jpg,0,0,0,0,1,0,0
4,02-2.jpg,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
12633,lrudNone_20230510151910_3_R_1.jpg,0,1,0,0,0,0,0
12634,None_20230510151910_4_R_1.jpg,0,1,0,0,0,0,0
12635,lrNone_20230510151910_4_R_1.jpg,0,1,0,0,0,0,0
12636,udNone_20230510151910_4_R_1.jpg,0,1,0,0,0,0,0


In [8]:
import random
ok_df = df[df['good']>0]

ok_df = ok_df.reset_index(drop=True)

rand_num_list = random.sample(range(len(ok_df)), 2500)
print(len(rand_num_list))
move_ok_df = ok_df.loc[ok_df.index.isin(rand_num_list)]

m_bar = tqdm.tqdm(move_ok_df.itertuples(), total=len(move_ok_df))
for rows in m_bar:
    new_row = pd.Series([rows.id, rows.good, rows.b_edge, rows.burr, rows.borken, rows.b_bubble, rows.etc, rows.no_lens], index=new_df.columns)
    new_df = new_df.append(new_row, ignore_index=True)
    if os.path.isfile(target_path + rows.id) is False and save_img:
        shutil.copy(base_path + rows.id, target_path + rows.id)
new_df

2500


100%|██████████| 2500/2500 [00:04<00:00, 501.97it/s]


,id,good,b_edge,burr,borken,b_bubble,etc,no_lens
0,01-2.jpg,0,0,0,0,1,0,0
1,lr01-2.jpg,0,0,0,0,1,0,0
2,ud01-2.jpg,0,0,0,0,1,0,0
3,lrud01-2.jpg,0,0,0,0,1,0,0
4,02-2.jpg,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
15133,None_20230508115324_0_L_1.jpg,1,0,0,0,0,0,0
15134,None_20230508115643_4_R_1.jpg,1,0,0,0,0,0,0
15135,None_20230508115733_2_R_1.jpg,1,0,0,0,0,0,0
15136,None_20230508115805_2_L_1.jpg,1,0,0,0,0,0,0


In [9]:
new_df.sum()

id          01-2.jpglr01-2.jpgud01-2.jpglrud01-2.jpg02-2.j...
good                                                     2500
b_edge                                                   3144
burr                                                      996
borken                                                   1316
b_bubble                                                 5156
etc                                                      2510
no_lens                                                     0
dtype: object

In [10]:
new_df.to_csv("lens_cls_dataset.csv",index=False)